# HMM Simulation

In [1]:
import numpy as np

In the upcoming HMM simulation we are going to run a "toy model" that will answer two simple questions, given the mood of a certain person over n days (when we know that his mood is affected by the weather):
1. What is the probability of getting such a mood series?
2. What is the most likely weather series?

## Modeling
First, here is a visual model that represents the transition probabilities between different states:
<img src="model.png" alt= “” width="500" height="500">


When the moods will be marked with 0 for sad and 1 for happy and will be called "observations", and the weather will be marked with 0 for rain and 1 for sun and will be called "states".
<br/><br/>
The transition matrix between the states will be called A when $ A_{ij} $ is $ P(X_j|X_i)$ ,$X = \{rain, sun\}$, <br/>
and the transition matrix between states & observations will be called A when $ B_{ij} $ is $ P(Y_j|X_i)$ ,$Y = \{sad, happy\}$.

In [2]:
A = np.array([ 
    [0.5,0.5],   
    [0.3,0.7]   
]) 
B = np.array([
    [0.8,0.2],        
    [0.4,0.6]    
])

pi is an array of probabilities when $ pi_i = P(X_i)$ and will be calculated using a Monte Carlo simulation that goes through the different situations a large number of times (the initial probability is completely random and can come in other forms).

In [3]:
pi = np.ones((A.shape[0]))/(A.shape[0])
for i in range(100):
    pi = pi @ A

The sequence of the result is the observations on the person, in our case it is about three days where in the first two he was sad and on the last day he was happy.

In [4]:
resSeq = [0,0,1]

## Forward algorithm
The forward algorithm is using to answer the first question (what is the probability of getting such a mood series?) and doing this by dynamic programming. <br/>
We create $T_{|Observations|x|Num Of States|}$ matrix when $T_{ij}$ is the probability of receiving the sequence we entered before until day i, when the state in day i is j (and his symbol is $a_i(X_j)$). <br/><br/>
These are the update rules:
1. $a_1(X_i) = Pi[i]*P(observations_1 | X_i)$ 
2. $a_t(X_i) =  \sum_{j=1}^{n} a_{t-1}(X_j)P(observations_t | X_i)P(X_i | X_j)$ 
3. $P(observations) = \sum_{i=1}^{n} a_{t}(X_i)$

In [5]:
Table = np.zeros((len(resSeq),A.shape[0]))

#Initialization
Table[0,:] = pi[:] * B[:,resSeq[0]]
    
#Update rules
for t in range(1,Table.shape[0]):
    for i in range(Table.shape[1]):
        for j in range(Table.shape[1]):
            Table[t,i] += Table[t-1,j]*A[j,i]*B[i,resSeq[t]] 

By the third rule we can compute $P(observations)$

In [6]:
np.sum(Table[-1,:])

0.1343999999999999

## Viterbi
Viterbi algorithm is an algorithm we will use to answer the second question (what is the most likely weather series?).The algorithm will use two tables in size $|NumberOfStates|x|NumberOfObservations|$ when $T_{1ij}$ represents the probability of the best path to state i at time j and $T_{2ij}$ represents the last state j in time i that gives the best probability. <br/>
The update rules are:
1. $T_{1ij} = max_k(T_1[k,j-1]A_{ki})B_{iy_j}$
2. $T_{1ij} = argmax_k(T_1[k,j-1]A_{ki})$

Then we run backward detection to reach the row of the most probable hidden states.

In [7]:
#Initialization
T1 = np.zeros((A.shape[0],len(resSeq)))
T2 = np.zeros((A.shape[0],len(resSeq)))

T1[:, 0] = pi * B[:, resSeq[0]]

#Update rules
for t in range(1, len(resSeq)):
    for j in range(A.shape[0]):
        val = T1[:, t - 1] * A[:, j]
        T1[j, t] = B[j, resSeq[t]] * np.max(val)
        T2[j, t] = np.argmax(val)
        
z_T = np.argmax(T1[:, len(resSeq) - 1])
X = [0] * len(resSeq)
X[-1] = z_T

for t in range(len(resSeq) - 2, -1, -1):
    z_t = T2[int(z_T), t + 1]
    X[t] = int(z_t)
    z_T = z_t

And here we see that the most likely situation is two days of rain followed by a sunny day.

In [8]:
X

[0, 0, 1]